In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv("../input/suv-data/suv_data.csv")
data.head()

In [ ]:
# Dropping "User ID"
data.drop(columns = ["User ID"], axis = 1, inplace = True)

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.nunique()

In [ ]:
data["Purchased"].value_counts()

In [ ]:
data.isna().sum()

In [ ]:
df = data.copy()

In [ ]:
df.corr()

In [ ]:
# Dropping "Gender" column
df.drop(["Gender"],axis=1, inplace = True)

In [ ]:
df.head()

In [ ]:
# Splitting the train test data
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(["Purchased"],axis=1)
y = df["Purchased"]

In [ ]:
# Scaling the data
from sklearn.preprocessing import StandardScaler

In [ ]:
scale = StandardScaler()

In [ ]:
X = scale.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=9)

## Random Forest model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=500, random_state=9)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
y_pred_tr = clf.predict(X_train)
y_pred = clf.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_train, y_pred_tr)
precision = precision_score(y_train, y_pred_tr)
recall = recall_score(y_train, y_pred_tr)

In [ ]:
print("Accuracy for train: ", accuracy)
print("Precision for train: ", precision)
print("Recall for train: ", recall)

In [ ]:
confusion_matrix(y_pred, y_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

In [ ]:
print("Accuracy for test: ", accuracy)
print("Precision for test: ", precision)
print("Recall for test: ", recall)

## Logistic Regression model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr_classifier = LogisticRegression(max_iter=500, random_state=9)

In [ ]:
lr_classifier.fit(X_train, y_train)

In [ ]:
y_pred_tr = clf.predict(X_train)

In [ ]:
accuracy = accuracy_score(y_train, y_pred_tr)
precision = precision_score(y_train, y_pred_tr)
recall = recall_score(y_train, y_pred_tr)

In [ ]:
print("Accuracy for train: ", accuracy)
print("Precision for train: ", precision)
print("Recall for train: ", recall)

In [ ]:
pred_ts = lr_classifier.predict(X_test)

In [ ]:
confusion_matrix(pred_ts, y_test)

In [ ]:
accuracy = accuracy_score(y_test, pred_ts)
precision = precision_score(y_test, pred_ts,labels=np.unique(pred_ts))
recall = recall_score(y_test, pred_ts,labels=np.unique(pred_ts))

In [ ]:
print("Accuracy for test: ", accuracy)
print("Precision for test: ", precision)
print("Recall for test: ", recall)

## XG Boost model

In [ ]:
from xgboost.sklearn import XGBClassifier

In [ ]:
xg_reg = XGBClassifier(use_label_encoder=False) 

In [ ]:
xg_reg.fit(X_train, y_train) 

In [ ]:
y_pred_xg = xg_reg.predict(X_test) 

In [ ]:
confusion_matrix(y_pred_xg, y_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred_xg)
precision = precision_score(y_test, y_pred_xg)
recall = recall_score(y_test, y_pred_xg)

In [ ]:
print("Accuracy for test: ", accuracy)
print("Precision for test: ", precision)
print("Recall for test: ", recall)

### Hyperparameter tuning of XGBoost Classifier

In [ ]:
clf = XGBClassifier(objective="binary:logistic",use_label_encoder=False)

In [ ]:
booster=['gbtree']
base_score=[0.1,0.3, 0.5]

In [ ]:
n_estimators = [100, 200]
max_depth = [4, 5]
learning_rate=[0.1,0.2]

In [ ]:
# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'booster':booster,
    'base_score':base_score
    }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Set up the random search with 3-fold cross validation
random_cv = RandomizedSearchCV(estimator=clf,
            param_distributions=hyperparameter_grid,
            cv=3, n_iter=10,
            scoring = 'accuracy',n_jobs = 3,
            verbose = 5, 
            return_train_score = True,
            random_state=24)

In [ ]:
random_cv.fit(X_train,y_train)

In [ ]:
random_cv.best_estimator_

In [ ]:
classifier = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None,use_label_encoder=False,objective="binary:logistic")

In [ ]:
classifier.fit(X_train,y_train)

In [ ]:
y_pred_xgb1 = classifier.predict(X_train)

In [ ]:
accuracy = accuracy_score(y_train, y_pred_xgb1)
precision = precision_score(y_train, y_pred_xgb1)
recall = recall_score(y_train, y_pred_xgb1)

In [ ]:
print("Accuracy for train: ", accuracy)
print("Precision for train: ", precision)
print("Recall for train: ", recall)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
confusion_matrix(y_pred, y_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

In [ ]:
print("Accuracy for test: ", accuracy)
print("Precision for test: ", precision)
print("Recall for test: ", recall)